In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# Load Data Frames into Pandas

In [2]:
obs_df = pd.read_csv('observations.csv')
species_df = pd.read_csv('species_info.csv')

In [3]:
print(obs_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB
None


# Data Frame Inspection

Here we see that there are a number of null objects within the convservation_status column. Looking at the csv file, we see that any null cell indicatest that they are "not endangered". Change non-null cells to "not-endangered" category"

In [4]:
print(species_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB
None


## Categorizing null values as "Not Endangered"

In [5]:
species_df = species_df.fillna("Not_Endangered" )

In [6]:
print(species_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  5824 non-null   object
dtypes: object(4)
memory usage: 182.1+ KB
None


In [7]:
print(species_df.describe())

              category scientific_name        common_names conservation_status
count             5824            5824                5824                5824
unique               7            5541                5504                   5
top     Vascular Plant   Procyon lotor  Brachythecium Moss      Not_Endangered
freq              4470               3                   7                5633


In [9]:
species_list = np.array(species_df.category.unique())
species_list

array(['Mammal', 'Bird', 'Reptile', 'Amphibian', 'Fish', 'Vascular Plant',
       'Nonvascular Plant'], dtype=object)

In [10]:
conservation_status = np.array(species_df.conservation_status.unique())
conservation_status

array(['Not_Endangered', 'Species of Concern', 'Endangered', 'Threatened',
       'In Recovery'], dtype=object)

In [11]:
print(obs_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB
None


Describing the observation data frame below does not tell us much. However, looking at the csv file, it will be useful to know the unique parks that are in the data set.

In [12]:
print(obs_df.describe())

       observations
count  23296.000000
mean     142.287904
std       69.890532
min        9.000000
25%       86.000000
50%      124.000000
75%      195.000000
max      321.000000


In [15]:
park_list = np.array(obs_df.park_name.unique())
park_list

array(['Great Smoky Mountains National Park', 'Yosemite National Park',
       'Bryce National Park', 'Yellowstone National Park'], dtype=object)